# **Chapter 16: Natural Language Processing with RNNs and Attention**

## **1. Pendahuluan**

Alan Turing pernah membayangkan sebuah tes untuk mengukur kecerdasan mesin, dan menariknya, ia memilih tugas linguistik (bahasa). Ini menunjukkan bahwa penguasaan bahasa adalah salah satu puncak kemampuan kognitif.

Dalam Machine Learning, **Natural Language Processing (NLP)** adalah bidang yang berfokus pada interaksi antara komputer dan bahasa manusia.

Bab ini membahas evolusi teknik NLP modern:
* **Character-RNN (Char-RNN):** Memprediksi karakter berikutnya dalam kalimat untuk menghasilkan teks (misalnya, meniru gaya Shakespeare).
* **Sentiment Analysis:** Mengklasifikasikan teks (misalnya, ulasan film) menjadi positif atau negatif.
* **Neural Machine Translation (NMT):** Menerjemahkan bahasa menggunakan arsitektur **Encoder-Decoder**.
* **Attention Mechanisms:** Mengatasi keterbatasan RNN tradisional dengan membiarkan model "fokus" pada bagian input tertentu.
* **Transformer:** Arsitektur revolusioner yang sepenuhnya membuang RNN dan hanya mengandalkan mekanisme Attention (dasar dari model seperti BERT dan GPT).

## **2. Generating Text dengan Character-RNN**

### **Konsep Dasar**
Ide dasarnya sederhana: berikan mesin serangkaian karakter, dan minta ia memprediksi karakter berikutnya. Jika kita melakukan ini berulang kali, kita bisa menghasilkan teks baru karakter demi karakter.

Misalnya, jika inputnya "Hell", model harus memprediksi "o" untuk membentuk "Hello".

### **Mempersiapkan Data**
Langkah pertama adalah mengubah teks menjadi angka. Kita menggunakan *character-level tokenizer*. Setiap karakter unik dipetakan ke integer tertentu.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Mengunduh teks Shakespeare
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

# Tokenisasi pada level karakter
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

# Mengubah teks menjadi urutan angka
max_id = len(tokenizer.word_index) # jumlah karakter unik
dataset_size = tokenizer.document_count # total karakter
encoded = np.array(tokenizer.texts_to_sequences([shakespeare_text]))[0]

print(f"Total karakter unik: {max_id}")
print(f"Contoh encoded text: {encoded[:10]}")
print(f"Mapping balik: {tokenizer.sequences_to_texts([encoded[:10]])}")

### **Membangun dan Melatih Model Char-RNN**
Untuk memprediksi karakter berikutnya, kita memotong teks menjadi jendela-jendela (*windows*) urutan input dan target.
* **Input:** Karakter ke-1 sampai ke-n.
* **Target:** Karakter ke-2 sampai ke-(n+1).

Kita menggunakan layer **GRU (Gated Recurrent Unit)** karena lebih efisien daripada LSTM namun tetap mampu menangkap pola jangka panjang.

In [ ]:
# Membuat dataset tf.data untuk pelatihan yang efisien
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1 # target = input digeser 1 langkah ke depan
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.shuffle(10000).batch(32)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

# Membangun Model
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), # Dropout untuk regularisasi
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# model.fit(dataset, epochs=5) # Komentar: Pelatihan memakan waktu lama
print("Model Char-RNN siap dilatih.")
model.summary()

## **3. Sentiment Analysis**

Selain menghasilkan teks, RNN sangat berguna untuk mengklasifikasikan urutan teks. Contoh klasiknya adalah **Sentiment Analysis** pada ulasan film IMDB (Positif vs Negatif).

### **Preprocessing Data Teks**
Berbeda dengan Char-RNN, di sini kita biasanya bekerja pada level **kata (word-level)**. Namun, alih-alih one-hot encoding (yang akan sangat besar dimensinya), kita menggunakan **Embeddings**.
* **Word Embedding:** Representasi vektor padat (dense vector) di mana kata-kata dengan makna serupa memiliki representasi vektor yang berdekatan.

In [ ]:
# Memuat dataset IMDB
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

# Word Index
word_index = keras.datasets.imdb.get_word_index()
# Menyesuaikan indeks (karena 3 indeks pertama dicadangkan)
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

print("Contoh ulasan ter-decode:")
print(" ".join([id_to_word[id_] for id_ in X_train[0]]))

# Membangun Model Klasifikasi
# Kita menggunakan layer Embedding dan GRU
model_sentiment = keras.models.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=128), # Vocab size 10000
    keras.layers.GRU(128, return_sequences=False),
    keras.layers.Dense(1, activation="sigmoid") # Output biner (Positif/Negatif)
])

model_sentiment.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print("\nArsitektur Model Sentimen:")
model_sentiment.summary()

## **4. Neural Machine Translation (NMT) dengan Encoder-Decoder**

Untuk menerjemahkan kalimat (misal: Inggris ke Spanyol), panjang input dan output bisa berbeda. RNN biasa tidak bisa menangani ini secara langsung. Kita membutuhkan arsitektur **Encoder-Decoder**.

### **Konsep Inti**
1.  **Encoder:** RNN yang membaca kalimat input urut dan meringkasnya menjadi satu vektor status (*context vector*).
2.  **Decoder:** RNN yang mengambil vektor status tersebut dan menghasilkan kalimat terjemahan langkah demi langkah.



[Image of Encoder-Decoder Diagram]


Ini sering disebut sebagai model **Seq2Seq**.

In [ ]:
import tensorflow_addons as tfa # Library tambahan sering digunakan untuk seq2seq

# Parameter dummy
vocab_size = 1000
embed_size = 128
units = 512

# Encoder
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
encoder_embeddings = keras.layers.Embedding(vocab_size, embed_size)(encoder_inputs)
# return_state=True agar kita bisa mengambil hidden state terakhir
encoder_outputs, state_h, state_c = keras.layers.LSTM(units, return_state=True)(encoder_embeddings)
encoder_state = [state_h, state_c] # Context Vector

# Decoder
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_embeddings = keras.layers.Embedding(vocab_size, embed_size)(decoder_inputs)
# Initial state decoder diisi dengan state terakhir encoder
decoder_lstm_layer = keras.layers.LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm_layer(decoder_embeddings, initial_state=encoder_state)
decoder_dense = keras.layers.Dense(vocab_size, activation="softmax")
output = decoder_dense(decoder_outputs)

model_nmt = keras.models.Model([encoder_inputs, decoder_inputs], output)
model_nmt.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
print("Model NMT Sederhana:")
model_nmt.summary()

**Penting:**
Saat pelatihan, kita memberikan target yang benar ke decoder (teknik ini disebut *Teacher Forcing*). Namun, saat inferensi (penggunaan nyata), decoder harus menggunakan prediksi langkah sebelumnya sebagai input langkah berikutnya.

## **5. Attention Mechanisms**

### **Masalah pada Encoder-Decoder Biasa**
Memaksa Encoder meringkas seluruh kalimat menjadi *satu* vektor status itu sulit, terutama untuk kalimat panjang. Informasi awal sering terlupakan (Bottleneck problem).

### **Solusi: Attention**
Mekanisme Attention mengizinkan Decoder untuk "melihat kembali" (memperhatikan) seluruh urutan output Encoder pada setiap langkah waktu, bukan hanya status terakhirnya. Decoder belajar untuk memberikan **bobot** (fokus) yang berbeda pada kata-kata input yang berbeda saat menghasilkan kata output tertentu.

* **Bahdanau Attention (Additive):** Menggunakan neural network kecil untuk menghitung skor alignment.
* **Luong Attention (Multiplicative):** Menggunakan dot product untuk menghitung kesamaan.

Di Keras, kita bisa menggunakan layer `tf.keras.layers.Attention` (Luong) atau `tf.keras.layers.AdditiveAttention` (Bahdanau).

In [ ]:
# Menambahkan Attention ke NMT
# Encoder (sekarang return_sequences=True karena Attention butuh semua output)
encoder_outputs, state_h, state_c = keras.layers.LSTM(units, return_sequences=True, return_state=True)(encoder_embeddings)
encoder_state = [state_h, state_c]

# Decoder
decoder_lstm_layer = keras.layers.LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm_layer(decoder_embeddings, initial_state=encoder_state)

# Attention Layer
# Menghubungkan output Decoder (query) dengan output Encoder (value/key)
attention_layer = keras.layers.Attention()
context_vector = attention_layer([decoder_outputs, encoder_outputs])

# Menggabungkan context vector dengan output decoder
decoder_concat_input = keras.layers.Concatenate()([decoder_outputs, context_vector])
output = keras.layers.TimeDistributed(keras.layers.Dense(vocab_size, activation="softmax"))(decoder_concat_input)

model_attention = keras.models.Model([encoder_inputs, decoder_inputs], output)
print("Model NMT dengan Attention:")
model_attention.summary()

## **6. The Transformer Architecture**

Pada tahun 2017, paper *"Attention Is All You Need"* mengubah segalanya. Mereka memperkenalkan **Transformer**, arsitektur yang:
1.  **Tidak menggunakan RNN sama sekali** (tidak ada recurrence).
2.  Mengandalkan sepenuhnya pada **Self-Attention** dan Feed-Forward Networks.
3.  Bisa diproses secara paralel (lebih cepat dilatih daripada RNN).

### **Komponen Kunci:**
* **Positional Encoding:** Karena tidak ada RNN, model tidak tahu urutan kata. Kita menyuntikkan informasi posisi secara matematis (menggunakan fungsi sinus/cosinus) ke dalam embedding input.
* **Multi-Head Attention:** Model memperhatikan input dari beberapa "perspektif" representasi yang berbeda secara bersamaan.

In [ ]:
# Implementasi Positional Encoding
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, **kwargs):
        super().__init__(**kwargs)
        self.max_steps = max_steps
        self.max_dims = max_dims
    
    def call(self, inputs):
        # Membuat encoding sinus/cosinus
        # (Implementasi detail rumus matematis sesuai paper)
        if self.max_dims % 2 == 1: self.max_dims += 1 # genapkan
        p, i = np.meshgrid(np.arange(self.max_steps), np.arange(self.max_dims // 2))
        pos_emb = np.empty((1, self.max_steps, self.max_dims))
        pos_emb[0, :, 0::2] = np.sin(p / 10000**(2 * i / self.max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / self.max_dims)).T
        return inputs + pos_emb[:, :tf.shape(inputs)[1], :]

# Visualisasi Positional Encoding
import matplotlib.pyplot as plt

max_steps = 200
max_dims = 512
pos_emb = PositionalEncoding(max_steps, max_dims)(np.zeros((1, max_steps, max_dims)))
plt.pcolormesh(pos_emb[0], cmap='RdBu')
plt.xlabel('Depth')
plt.ylabel('Position')
plt.colorbar()
plt.title("Visualisasi Positional Encoding Matrix")
plt.show()

**Dampak:**
Arsitektur Transformer menjadi fondasi bagi model bahasa raksasa (LLM) modern seperti BERT, GPT-2, GPT-3, dan seterusnya. Kemampuannya menangani dependensi jarak jauh jauh lebih baik daripada LSTM.

## **7. Kesimpulan**

Dalam Chapter 16 ini, kita telah menjelajahi evolusi pemrosesan bahasa alami:
1.  **RNN & LSTM/GRU:** Dasar pemrosesan sekuens, mampu menangkap konteks temporal.
2.  **Encoder-Decoder:** Memungkinkan pemetaan urutan-ke-urutan (seq2seq) untuk penerjemahan.
3.  **Attention:** Terobosan yang mengatasi "lupa" pada kalimat panjang dengan membiarkan model fokus pada bagian relevan.
4.  **Transformer:** Standar emas saat ini yang menggunakan *pure attention* untuk efisiensi dan performa tinggi.

Pemahaman tentang mekanisme Attention adalah kunci untuk memahami perkembangan terkini dalam Deep Learning.